In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
train = pd.read_csv("train_final.csv", index_col=['date'], parse_dates=['date'])
test = pd.read_csv("test_final.csv", index_col=['date'], parse_dates=['date'])

In [3]:
x_train = train.drop(columns = 'new_confirmed')
y_train = train.new_confirmed
x_test = test.drop(columns = 'new_confirmed')
y_test = test.new_confirmed

In [4]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import prophet

## Prophet Single


In [23]:
model = prophet.Prophet().fit(train.reset_index().rename(columns={'date':'ds', 'new_confirmed':'y'}))

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp1a8r8vfy/tpzbjpyp.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp1a8r8vfy/ib9vfjiy.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=40574', 'data', 'file=/tmp/tmp1a8r8vfy/tpzbjpyp.json', 'init=/tmp/tmp1a8r8vfy/ib9vfjiy.json', 'output', 'file=/tmp/tmp1a8r8vfy/prophet_modelpuq_bink/prophet_model-20231111050544.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
05:05:44 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
05:06:12 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [24]:
train_pred=model.predict(train.reset_index().rename(columns={'date':'ds', 'new_confirmed':'y'})).set_index('ds')
train_pred.head()

,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
ds,,,,,,,,,,,,,,,,,,
2020-01-01,456.955859,-3680.319912,2596.439893,456.955859,456.955859,-1052.24957,-1052.24957,-1052.24957,83.894805,83.894805,83.894805,-1136.144375,-1136.144375,-1136.144375,0.0,0.0,0.0,-595.293711
2020-01-01,456.955859,-3645.190008,2382.462599,456.955859,456.955859,-1052.24957,-1052.24957,-1052.24957,83.894805,83.894805,83.894805,-1136.144375,-1136.144375,-1136.144375,0.0,0.0,0.0,-595.293711
2020-01-01,456.955859,-3684.933747,2702.005891,456.955859,456.955859,-1052.24957,-1052.24957,-1052.24957,83.894805,83.894805,83.894805,-1136.144375,-1136.144375,-1136.144375,0.0,0.0,0.0,-595.293711
2020-01-01,456.955859,-3705.143165,2586.509693,456.955859,456.955859,-1052.24957,-1052.24957,-1052.24957,83.894805,83.894805,83.894805,-1136.144375,-1136.144375,-1136.144375,0.0,0.0,0.0,-595.293711
2020-01-01,456.955859,-3592.674454,2533.113562,456.955859,456.955859,-1052.24957,-1052.24957,-1052.24957,83.894805,83.894805,83.894805,-1136.144375,-1136.144375,-1136.144375,0.0,0.0,0.0,-595.293711


In [27]:
train_pred_date = pd.DataFrame(train_pred.index)
pd.concat([train_pred_date,train_pred.yhat], axis=1).plot(figsize=(15,5))
plt.legend(['train', 'train_pred'])

<ipython-input-27-437ce579893f>:2: RuntimeWarning: '<' not supported between instances of 'int' and 'Timestamp', sort order is undefined for incomparable objects.
  pd.concat([train_pred_date,train_pred.yhat], axis=1).plot(figsize=(15,5))


InvalidIndexError: ignored

In [ ]:
test_pred=model.predict(test.reset_index().rename(columns={'date':'ds', 'new_confirmed':'y'})).set_index('ds')
pd.concat([test,test_pred['yhat']], axis=1).plot( )
plt.legend(['test', 'test_pred'])

## Prophet Multi


In [ ]:
def process(df):
    lag1df = df.shift(1)
    lag1df.columns = ['lag1_' + str(col) for col in lag1df.columns]
    df=df[['new_confirmed']].join(lag1df)
    return df.fillna(0)
train=process(train)
test=process(test)
print(train.shape, test.shape)

In [ ]:
model = prophet.Prophet()
for col in train.columns:
    if col not in ['date', 'new_confirmed']:
        model.add_regressor(col)
model=model.fit(train.reset_index().rename(columns={'date':'ds', 'total_purchase_amt':'y'}))

In [ ]:
train_pred=model.predict(train.reset_index().rename(columns={'date':'ds', 'total_purchase_amt':'y'})).set_index('ds')
train_pred.head()

In [ ]:
pd.concat([train['total_purchase_amt'],train_pred['yhat']], axis=1).plot(figsize=(15,5))
plt.legend(['train', 'train_pred'])

In [ ]:
test_pred=model.predict(test.reset_index().rename(columns={'date':'ds', 'total_purchase_amt':'y'})).set_index('ds')
pd.concat([test['total_purchase_amt'],test_pred['yhat']], axis=1).plot( )
plt.legend(['test', 'test_pred'])